In [1]:
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscorescoringv2

import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import time as time
from time import sleep
from IPython.core.display import clear_output
import sqlite3

In [2]:
def season_string(season):
    return str(season) + '-' + str(season+1)[-2:]

In [3]:
def add_basic_boxscores(conn, start_season, end_season, if_exists='append'):
    """This function pulls basic team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_basic_boxscores in the sqlite db"""
    
    cur = conn.cursor()
    table_name = 'team_basic_boxscores'
    
    if if_exists == 'replace':
        cur.execute('DROP TABLE IF EXISTS ' + table_name)
        cur.execute('VACUUM')
        
    cur.execute("""CREATE TABLE IF NOT EXISTS {} (SEASON TEXT, TEAM_ID INTEGER, TEAM_ABBREVIATION TEXT, 
        TEAM_NAME TEXT, GAME_ID TEXT, GAME_DATE DATE, MATCHUP TEXT, WL TEXT, MIN INTEGER, FGM INTEGER, FGA INTEGER, 
        FG_PCT FLOAT, FG3M INTEGER, FG3A  INTEGER, FG3_PCT FLOAT, FTM INTEGER, FTA INTEGER, FT_PCT FLOAT, OREB INTEGER,
        DREB INTEGER, REB INTEGER, AST INTEGER, STL INTEGER, BLK INTEGER, TOV INTEGER, PF INTEGER, PTS INTEGER, 
        PLUS_MINUS INTEGER)""".format(table_name))    
    
    for season in tqdm(range(start_season, end_season+1)):
        season_str = season_string(season)
        season_boxscores = []

        for season_type in ['Regular Season', 'Playoffs']:
            boxscores = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            season_boxscores.append(boxscores)
            sleep(2)
        season_df = pd.concat(season_boxscores)
        season_df['SEASON'] = season_str
        season_df.drop(columns = ['SEASON_ID', 'VIDEO_AVAILABLE'], inplace=True)
        
        season_df.to_sql(table_name, conn, if_exists='append', index=False)
        
        sleep(3)
        
    #cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return None

In [4]:
def add_advanced_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls advanced team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_advanced_boxscores in the sqlite db
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    table_name = 'team_advanced_boxscores'
    game_ids_not_added = []
    
    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS {} (GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, 
        TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT, MIN TEXT, E_OFF_RATING FLOAT, OFF_RATING FLOAT, E_DEF_RATING FLOAT, 
        DEF_RATING FLOAT, E_NET_RATING FLOAT, NET_RATING FLOAT, AST_PCT FLOAT, AST_TOV FLOAT, 
        AST_RATIO FLOAT, OREB_PCT FLOAT, DREB_PCT FLOAT, REB_PCT FLOAT, E_TM_TOV_PCT FLOAT, 
        TM_TOV_PCT FLOAT, EFG_PCT FLOAT, TS_PCT FLOAT, USG_PCT FLOAT, E_USG_PCT FLOAT, E_PACE FLOAT, 
        PACE FLOAT, PACE_PER40 FLOAT, POSS FLOAT, PIE FLOAT)'''.format(table_name))
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)
        season_team_boxscores = []

        for season_type in ['Regular Season', 'Playoffs']:
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        team_boxscores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id).get_data_frames()[1]                    
                        team_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)
        
        sleep(120)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    print(len(game_ids_not_added), ' games not added')
    return game_ids_not_added

In [5]:
def add_scoring_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls scoring team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_scoring_boxscores in the sqlite db.
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    table_name = 'team_scoring_boxscores'
    game_ids_not_added = []

    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
    
    conn.execute('''GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT,
       MIN TEXT, PCT_FGA_2PT FLOAT, PCT_FGA_3PT FLOAT, PCT_PTS_2PT FLOAT, PCT_PTS_2PT_MR FLOAT,
       PCT_PTS_3PT FLOAT, PCT_PTS_FB FLOAT, PCT_PTS_FT FLOAT, PCT_PTS_OFF_TOV FLOAT,
       PCT_PTS_PAINT FLOAT, PCT_AST_2PM FLOAT, PCT_UAST_2PM FLOAT, PCT_AST_3PM FLOAT,
       PCT_UAST_3PM FLOAT, PCT_AST_FGM FLOAT, PCT_UAST_FGM FLOAT)'''.format(table_name))
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)
        season_team_boxscores = []

        for season_type in ['Regular Season', 'Playoffs']:
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        scoring_boxscores = boxscorescoringv2.BoxScoreScoringV2(game_id).get_data_frames()[1]
                        scoring_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)
        print(season_str, ' completed')
        sleep(120)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return game_ids_not_added

In [6]:
## Update basic team gamelogs and player gamelogs

def update_team_basic_boxscores(conn, season):
    table_name = 'team_basic_boxscores'
    season_str = season_string(season)
        
    dfs = []
    for season_type in ['Regular Season', 'Playoffs']:
        team_gamelogs = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
        dfs.append(team_gamelogs)
        
    team_gamelogs_updated = pd.concat(dfs)
    team_gamelogs_updated['SEASON'] = season_str
    team_gamelogs_updated.drop(columns = ['SEASON_ID', 'VIDEO_AVAILABLE'], inplace=True)
    
    team_gamelogs_updated.to_sql(table_name, conn, if_exists='append', index=False)

    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return None

In [7]:
def update_team_advanced_boxscores(conn, season, dates):
    table_name = 'team_advanced_boxscores'
    
    season_str = season_string(season)
    
    game_ids_not_added = []
    
    # Pull the GAME_IDs from my data
    game_ids_in_db = pd.read_sql('''SELECT DISTINCT team_basic_boxscores.GAME_ID FROM team_basic_boxscores
                INNER JOIN team_advanced_boxscores 
                ON team_basic_boxscores.GAME_ID = team_advanced_boxscores.GAME_ID
                AND team_basic_boxscores.TEAM_ID = team_advanced_boxscores.TEAM_ID
                WHERE SEASON = "{}" '''.format(season_str), conn)

    game_ids_in_db = game_ids_in_db['GAME_ID'].tolist()
    
    missing_game_ids = []
    if len(dates) != 0:
        for date in dates:
            gamelogs = leaguegamelog.LeagueGameLog(
                season=season_str, date_from_nullable=date, date_to_nullable=date).get_data_frames()[0]
            missing_game_ids.extend(gamelogs['GAME_ID'].unique())
            
    else:        
        # get up to date GAME_IDs
        to_date_game_ids = []
        for season_type in ['Regular Season', 'Playoffs']:
            to_date_gamelogs = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            to_date_game_ids.extend(to_date_gamelogs['GAME_ID'].unique())

        # See which game_ids are missing
        missing_game_ids = set(to_date_game_ids) - set(game_ids_in_db)
        
    num_games_updated = len(missing_game_ids)
    print("num_games_updated:", num_games_updated)
    
    if num_games_updated == 0:
        print("All team advanced boxscores up to date in season {}".format(season_str))
        return None
    
    for game_id in tqdm(missing_game_ids, desc='progress'):
        try:
            boxscores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id).get_data_frames()[1]
            boxscores.to_sql(table_name, conn, if_exists='append', index=False)
            sleep(2)
        except:
            game_ids_not_added.append(game_id)  
    
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT max(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return game_ids_not_added

In [8]:
def update_team_scoring_boxscores(conn, season, dates):
    table_name = 'team_scoring_boxscores'

    season_str = season_string(season)

    game_ids_not_added = []

    # Pull the GAME_IDs from my data
    game_ids_in_db = pd.read_sql(f'''SELECT DISTINCT team_scoring_boxscores.GAME_ID FROM team_basic_boxscores
                INNER JOIN team_scoring_boxscores 
                ON team_basic_boxscores.GAME_ID = team_scoring_boxscores.GAME_ID
                AND team_basic_boxscores.TEAM_ID = team_scoring_boxscores.TEAM_ID
                WHERE SEASON = "{season_str}" ''', conn)

    game_ids_in_db = game_ids_in_db['GAME_ID'].tolist()

    missing_game_ids = []
    if len(dates) != 0:
        for date in dates:
            gamelogs = leaguegamelog.LeagueGameLog(
                season=season_str, date_from_nullable=date, date_to_nullable=date).get_data_frames()[0]
            missing_game_ids.extend(gamelogs['GAME_ID'].unique())

    else:
        # get up to date GAME_IDs
        to_date_game_ids = []
        for season_type in ['Regular Season', 'Playoffs']:
            to_date_gamelogs = leaguegamelog.LeagueGameLog(
                season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            to_date_game_ids.extend(to_date_gamelogs['GAME_ID'].unique())

        # See which game_ids are missing
        missing_game_ids = set(to_date_game_ids) - set(game_ids_in_db)
        
    num_games_updated = len(missing_game_ids)
    print("num_games_updated:", num_games_updated)

    if num_games_updated == 0:
        print("All team advanced boxscores up to date in season {}".format(season_str))
        return None

    for game_id in tqdm(missing_game_ids, desc='progress'):
        try:
            boxscores = boxscorescoringv2.BoxScoreScoringV2(
                game_id).get_data_frames()[1]
            boxscores.to_sql(table_name, conn,
                             if_exists='append', index=False)
            sleep(2)
        except:
            game_ids_not_added.append(game_id)

    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT max(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(
        table_name, table_name))
    conn.commit()

    return game_ids_not_added

In [9]:
con = sqlite3.connect('../nba.db')

In [27]:
add_basic_boxscores(con, 2000, 2022)

100%|██████████| 23/23 [03:16<00:00,  8.52s/it]


In [10]:
team_basic_boxscores_df = pd.read_sql('select * from team_basic_boxscores', con)
team_basic_boxscores_df.head()

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,2021-22,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19,LAL vs. GSW,L,240,45,...,5,40,45,21,7,4,18,25,114,-7
1,2021-22,1610612744,GSW,Golden State Warriors,0022100002,2021-10-19,GSW @ LAL,W,240,41,...,9,41,50,30,9,2,17,18,121,7
2,2021-22,1610612751,BKN,Brooklyn Nets,0022100001,2021-10-19,BKN @ MIL,L,240,37,...,5,39,44,19,3,9,13,17,104,-23
3,2021-22,1610612749,MIL,Milwaukee Bucks,0022100001,2021-10-19,MIL vs. BKN,W,240,48,...,13,41,54,25,8,9,8,19,127,23
4,2021-22,1610612754,IND,Indiana Pacers,0022100003,2021-10-20,IND @ CHA,L,240,42,...,8,43,51,29,2,10,17,24,122,-1


In [11]:
pd.read_sql('select min(SEASON), max(SEASON) from team_basic_boxscores', con)

,min(SEASON),max(SEASON)
0,2000-01,2021-22


In [22]:
game_ids_excluded = add_advanced_boxscores(con, 2003, 2003)

4  games not added


In [12]:
pd.read_sql("""SELECT DISTINCT(b.SEASON)
               FROM team_advanced_boxscores as a
               LEFT JOIN team_basic_boxscores as b
               ON a.GAME_ID = b.GAME_ID AND a.TEAM_ID = b.TEAM_ID
            """, con)

,SEASON
0,2021-22
1,None
2,2020-21
3,2019-20
4,2018-19
5,2017-18
6,2016-17
7,2013-14
8,2012-13
9,2008-09


In [14]:
#concatenate advanced boxscore data
team_advanced_boxscores_main = pd.read_sql('select * from team_advanced_boxscores', con)
team_advanced_boxscores_1 = pd.read_csv('team_advanced_boxscores_151109060200.csv')
team_advanced_boxscores_2 = pd.read_csv('team_advanced_boxscores_1410070501.csv')

team_advanced_boxscores = pd.concat([team_advanced_boxscores_main, team_advanced_boxscores_1, team_advanced_boxscores_2])

In [20]:
print(len(team_advanced_boxscores))
team_advanced_boxscores.head()

56164


,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,Unnamed: 0
0,0022100002,1610612747,Lakers,LAL,Los Angeles,240:00,98.0,101.8,106.0,107.1,...,0.553,0.551,1.0,0.198,115.28,112.5,93.75,112.0,0.422,NaN
1,0022100002,1610612744,Warriors,GSW,Golden State,240:00,106.0,107.1,98.0,101.8,...,0.516,0.570,1.0,0.200,115.28,112.5,93.75,113.0,0.578,NaN
2,0022100001,1610612749,Bucks,MIL,Milwaukee,240:00,117.7,124.5,101.8,102.0,...,0.538,0.562,1.0,0.194,105.02,102.0,85.00,102.0,0.593,NaN
3,0022100001,1610612751,Nets,BKN,Brooklyn,240:00,101.8,102.0,117.7,124.5,...,0.542,0.552,1.0,0.200,105.02,102.0,85.00,102.0,0.407,NaN
4,0022100003,1610612754,Pacers,IND,Indiana,240:00,111.4,114.0,107.1,116.0,...,0.561,0.607,1.0,0.199,112.22,106.5,88.75,107.0,0.520,NaN


In [ ]:
print(team_advanced_boxscores[''])

In [32]:
#find missing games in advanced boxscore data
game_ids = team_basic_boxscores_df['GAME_ID'].unique()
advanced_boxscore_game_ids = team_advanced_boxscores['GAME_ID'].unique()
game_ids_str = game_ids.astype('str')
adv_game_ids_str = advanced_boxscore_game_ids.astype('str')
print(game_ids_str.dtype)
print(adv_game_ids_str.dtype)

<U10
<U10


In [36]:
missing_game_ids = np.setdiff1d(game_ids_str, adv_game_ids_str)
missing_game_ids_2 = np.setdiff1d(adv_game_ids_str, game_ids_str)
print(len(game_ids))
print(len(advanced_boxscore_game_ids))
print(len(missing_game_ids))
print(missing_game_ids)
print(len(missing_game_ids_2))
print(missing_game_ids_2)

28154
28082
14107
['0020000001' '0020000002' '0020000003' ... '0041500407' '0041700232'
 '0041800233']
14035
['0042100405' '20000001' '20000002' ... '41500405' '41500406' '41500407']


In [42]:
print(team_basic_boxscores_df[team_basic_boxscores_df['GAME_ID']=='0041500407'])

        SEASON     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME  \
43826  2015-16  1610612744               GSW  Golden State Warriors   
43827  2015-16  1610612739               CLE    Cleveland Cavaliers   

          GAME_ID   GAME_DATE      MATCHUP WL  MIN  FGM  ...  OREB  DREB  REB  \
43826  0041500407  2016-06-19  GSW vs. CLE  L  240   32  ...     7    32   39   
43827  0041500407  2016-06-19    CLE @ GSW  W  240   33  ...     9    39   48   

       AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  
43826   22    7    5   10  23   89          -4  
43827   17    7    6   11  15   93           4  

[2 rows x 28 columns]


In [24]:
team_advanced_boxscores_df = pd.read_sql('select * from team_advanced_boxscores', con)
team_advanced_boxscores_df.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022100002,1610612747,Lakers,LAL,Los Angeles,240:00,98.0,101.8,106.0,107.1,...,16.1,0.553,0.551,1.0,0.198,115.28,112.5,93.75,112.0,0.422
1,0022100002,1610612744,Warriors,GSW,Golden State,240:00,106.0,107.1,98.0,101.8,...,15.0,0.516,0.570,1.0,0.200,115.28,112.5,93.75,113.0,0.578
2,0022100001,1610612749,Bucks,MIL,Milwaukee,240:00,117.7,124.5,101.8,102.0,...,7.8,0.538,0.562,1.0,0.194,105.02,102.0,85.00,102.0,0.593
3,0022100001,1610612751,Nets,BKN,Brooklyn,240:00,101.8,102.0,117.7,124.5,...,12.7,0.542,0.552,1.0,0.200,105.02,102.0,85.00,102.0,0.407
4,0022100003,1610612754,Pacers,IND,Indiana,240:00,111.4,114.0,107.1,116.0,...,15.9,0.561,0.607,1.0,0.199,112.22,106.5,88.75,107.0,0.520


In [25]:
#team_basic_boxscores_df.to_csv('team_basic_boxscores.csv', mode='w+')
team_advanced_boxscores_df.to_csv('team_advanced_boxscores.csv', mode='w+')

In [ ]:
#need to go back and pull missing games for advanced and scoring boxscores